<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<h1>Algorísmica Avançada 2022</h1>
<h2>Laboratori 5 - Enumeratius</h2>
</center>
</div>

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<div class="alert alert-info">
<h2>Avís general per a les Pràctiques de Laboratori</h2>

És molt important <b>comentar tot el vostre codi</b>. Comentar el codi no només facilita a altres persones que el llegeixin la compressió del que esteu fent, sinó també a vosaltres mateixos durant el procés de disseny i implementació. D'aquesta forma podreu entendre el vostre propi codi encara que faci molt temps que no el consulteu. A més, a l'hora de ser corregits, un codi comentat s'entén molt millor, la qual cosa ajuda a traslladar al professor les idees que teníeu a l'intentar implementar-lo i encara més en el cas que no ho hagueu pogut acabar-ho amb èxit. No cal comentar cada línia, però si el que fa cada part.

En segon lloc, és molt important <b>respectar les capçaleres de les funcions i estructura de la funció ```return```</b> que s'indiquin a l'enunciat. Tot i que llegim i corregim els vostres codis en detall, fem servir correctors automàtics per guiar i agilitzar el procés i saber quines solucións funcionen i quines no ja d'entrada. 

Axí doncs, si es demana que la funció tingui la següent estructura: 

```python
def exercici1(param1, param2):
    # implementació de la solució
    return resultat 
```

És important que en el notebook existeixi aquesta funció i que no canvieu el nom dels paràmetres ni el tipus de dada retornada.
    
</div>




<div class="alert alert-success">
    <h1>Problema: Edificis</h1>
    <p>
        Volem solucionar el puzzle dels edificis. Donada una graella com la que tenim a continuació (esquerra), volem trobar la solució al puzzle (dreta):
        <center><img src="img/sky-intro.png" width='25%'></center>
        Podem pensar que cada una de les caselles és un solar on hi podem posar un edifici de X plantes. En aquest cas un edifici de 1,2,3 o 4 plantes ja que el tauler és de 4x4.<br>
        Els números que trobem a fora de la graella ens indiquen quants edificis són visibles des d'aquesta posició. Un edifici d'alçada 4 tapa qualsevol edifici que hi ha al darrere com podeu veure en la imatge següent:
        <center><img src="img/sky-rule-3-1.png" width='25%'></center>
        A més, en cada fila i columna de la graella només pot haver-hi un sol edifici de cada alçada, és a dir, no es poden repetir.
        <center><img src="img/skyscrapers.png" width='25%'></center>
        L'objectiu és doncs, donada una graella com la inicial, trobar la configuració d'edificis que satisfà les condicions. Implementeu un algorisme usant backtracking que <b>trobi una solució utilitzant el mínim nombre de crides recursives i minimitzant el temps total d'execució.</b>. Per fer-ho haureu de tenir en comptes diferents factors com ara:
        <ol>
            <li> Quin número començareu provant?
            <li> Quina casella serà la primera que omplireu?
        </ol>
    </p>
    
</div>

La funció **format_sky** us ajudarà a visualitzar el tauler del problema.

In [3]:
import numpy as np
from IPython.display import clear_output
import time

def format_sky(board, top, bottom, left, right):
    """
    Funció auxiliar per mostrar la matriu del Problema 1 en forma de graella.
    """
    _str = ""
    
    # Files
    for v in top:
        _str+="   "+str(v)
    _str = "  " + _str + "\n"

    for idx, i in enumerate(board):
        _str+= "   +"+ ("-"*((board.shape[1]*4)-1)) +"+\n"
        _str+= f" {left[idx]} | "
        
        # Columnes
        for j in i:   
            if j!=0:
                _str+= str(j)+" | "
            else:
                 _str+= "  | "
            
        _str+= f"{right[idx]} \n"
    _str+= "   +"+ ("-"*((board.shape[1]*4)-1))+"+\n"
    
    _bot = ""
    for v in bottom:
        _bot+="   "+str(v)
    _str += "  " + _bot
    
    return _str.replace('0',' ')

Veieu un exemple de com utilitzar aquesta funció:


In [4]:
# Amb aquesta funció es mostren totes les caselles del tauler
# Les caselles que contenen un zero es mostren buides

# Creem unes restriccions i un tauler 
top, bottom, left, right = [0,0,0], [3,0,0], [0,0,0], [0,0,1]
grid = np.zeros((len(left), len(top)), dtype='int')

print('Tauler:')
print(format_sky(grid, top, bottom, left, right))

Tauler:
              
   +-----------+
   |   |   |   |   
   +-----------+
   |   |   |   |   
   +-----------+
   |   |   |   | 1 
   +-----------+
     3        


La funció **delayed_print** us permet bloquejar temporalment un 'print' per si voleu veure les accions que fa a cada pas el vostre algorisme.<br>
No heu d'utilitzar-la a l'entrega ja que el programa tardarà molt més del necessari, utilitzeu-la només mentre programeu la solució.

In [5]:
def delayed_print(grid, top, bottom, left, right, sleep_time=0.02):
    """
    Funció auxiliar que bloqueja temporalment la funció 'print' per veure l'evolució de l'algorisme.
    """
    print(format_sky(grid, top, bottom, left, right))
    clear_output(wait=True)
    time.sleep(sleep_time)

In [6]:
top, bottom, left, right = [0,0,0], [3,0,0], [0,0,0], [0,0,1]
grid = np.zeros((len(left), len(top)), dtype='int')

for i in range(0, len(left)):
    for j in range(0, len(top)):
        grid[i][j] = np.random.randint(1,4)
        delayed_print(grid, top, bottom, left, right, sleep_time=0.2)

              
   +-----------+
   | 3 | 3 | 2 |   
   +-----------+
   | 3 | 3 | 3 |   
   +-----------+
   | 3 | 2 | 2 | 1 
   +-----------+
     3        


In [177]:
def next_cell_skyscraper(grid, constraints):
   
    for constraint in constraints:
        
        value, index, direction = constraint
        
        # Els que no tinguin restricció els tractarem més endevant
        if value == 0:
            continue

        if direction == "top":
            for j in range(len(grid)):
                if grid[j][index] == 0:
                    return j, index
                
        elif direction == "left":
            for j in range(len(grid)):
                if grid[index][j] == 0:
                    return index, j
        
        elif direction == "bottom":
            for j in range(len(grid) - 1, -1, -1):
                if grid[j][index] == 0:
                    return j, index
                               
        elif direction == "right":
            for j in range(len(grid) - 1, -1, -1):
                if grid[index][j] == 0:
                    return index, j
                
    for i in range(len(grid)):
        for j in range(len(grid)):
            if grid[i][j] == 0:
                return i, j
            
    return -1, -1


def satisfies(grid, x, y, num, top, bottom, left, right):
    """
    Donat un tauler, un punt amb coordenades (x,y) i un nombre 'num', comprova si és possible assignar-lo a la posició (x,y) 
    donades les restriccions del problema.
    
    Params
    ------
    :grid: Una matriu de numpy
    :x,y: Un punt de la matriu on volem posar el nombre 'num'
    :num: Un nombre enter
    :top, bottom, left, right: Els nombres de fora del tauler de la part superior, inferior, esquerra i dreta, respectivament.
    
    Returns
    -------
    :b: Un boleà True/False depenent de si el nombre 'num' pot ser col·locat a la casella (x,y) o no.
    """
    
    # Mirem si está en la mateixa fila o columna
    for i in range(len(grid)):
    
        if grid[i][y] == num or grid[x][i] == num:
            return False
    
    # Posem el numero en un grid comporvant que veiem menys o igual dels edificis necesaris
    aux = grid.copy()
    aux[x][y] = num 
    
    if top[y] != 0:
        
        visible = 0
        max_sky = 0
        for i in range(len(grid)):
            if aux[i][y] == 0:
                break
            if aux[i][y] > max_sky:
                visible += 1
                max_sky = aux[i][y]

        if visible > top[y]:
            #print("PROBLEMA CON TOP (visible > top[y]) para poner en x:" + str(x) + " y:" + str(y) +" el valor " + str(num))
            #print(format_sky(np.array(aux), top, bottom, left, right))
            return False
    #check left
    if left[x] != 0:
        
        visible = 0
        max_sky = 0

        for i in range(len(grid)):
            
            if aux[x][i] == 0:
                break
            
            if aux[x][i] > max_sky:
                visible += 1
                max_sky = aux[x][i]
                
                
        if visible > left[x]:
            #print("PROBLEMA CON LEFT (visible > left[x]) para poner en x:" + str(x) + " y:" + str(y) +" el valor " + str(num))
            #print(format_sky(np.array(aux), top, bottom, left, right))
            return False
        
    if bottom[y] != 0:
        
        visible = 0
        max_sky = 0
        for i in range(len(grid) -1, -1, -1):
            
            if aux[i][y] == 0:
                break
                
            if aux[i][y] > max_sky:
                visible += 1
                max_sky = aux[i][y]

        if visible > bottom[y]:
            #print("PROBLEMA CON TOP (visible > top[y]) para poner en x:" + str(x) + " y:" + str(y) +" el valor " + str(num))
            #print(format_sky(np.array(aux), top, bottom, left, right))
            return False
          
        
    #check left
    if right[x] != 0:
        
        visible = 0
        max_sky = 0

        for i in range(len(grid) - 1, -1, -1):
            if aux[x][i] == 0:
                break
                
            if aux[x][i] > max_sky:
                visible += 1
                max_sky = aux[x][i]
                
                
        if visible > right[x]:
            #print("PROBLEMA CON LEFT (visible > left[x]) para poner en x:" + str(x) + " y:" + str(y) +" el valor " + str(num))
            #print(format_sky(np.array(aux), top, bottom, left, right))
            return False


    
    #print("NINGÚN PROBLEMA para poner en x:" + str(x) + " y:" + str(y) +" el valor " + str(num))
    #print(format_sky(np.array(aux), top, bottom, left, right))
    aux[x][y] = 0
    
    return True


# Aquest métode comprova al final del arbre de backtracking si la fulla és realment una solució.
def itsSolved(grid, top, bottom, left, right):
    

    #CHECK TOP
    for i in range(len(grid)):
        if top[i] != 0:
            visible = 0
            max_sky = 0
            for j in range(len(grid)):
                if grid[j][i] > max_sky:
                    visible += 1
                    max_sky = grid[j][i]

            if visible != top[i]:
                return False
    
    #CHECK BOTTOM
    for i in range(len(grid)):
        if bottom[i] != 0:
            visible = 0
            max_sky = 0
            for j in range(len(grid) - 1, -1, -1):
                if grid[j][i] > max_sky:
                    visible += 1
                    max_sky = grid[j][i]

            if visible != bottom[i]:
                return False

    #CHECK LEFT
    for i in range(len(grid)):
        
        if left[i] != 0:
            visible = 0
            max_sky = 0
            for j in range(len(grid)):
                if grid[i][j] > max_sky:
                    visible += 1
                    max_sky = grid[i][j]

            if visible != left[i]:
                return False
    
    #CHECK RIGHT
    for i in range(len(grid)):
        
        if right[i] != 0:
            visible = 0
            max_sky = 0
            for j in range(len(grid) - 1, -1, -1):
                if grid[i][j] > max_sky:
                    visible += 1
                    max_sky = grid[i][j]

            if visible != right[i]:
                return False

     

    
    return True
     
def skyscrapper_backtracking(grid, top, bottom, left, right, constraints):  
    """
    Funció que implementa l'algorisme de backtracking
    
    Params
    ------
    :grid: Una matriu de tipus numpy
    :top, bottom, left, right: Els nombres de fora del tauler de la part superior, inferior, esquerra i dreta, respectivament.
    
    Returns
    -------
    :grid: La matriu plena amb la solució correcta o bé False en cas que no tingui solució.    
    """
    
    # NO esborreu aquestes dues línies, serveien per saber quants cops es crida aquesta funció
    # ----------------------------------------------------------------------------------------
    global num_recursive_calls 
    num_recursive_calls += 1
    # ----------------------------------------------------------------------------------------
    # Trobem la primera casella buida del tauler
    x, y = next_cell_skyscraper(grid, constraints)

    # Cas base: Si no hi ha cap casella buida, vol dir que hem acabat i retornem el tauler
    if x == -1 and y == -1:
        if(itsSolved(grid, top, bottom, left, right)):
            return True
        else:
            return False

    # Provoquem una recursió per cada nombre possible posant primer els edifics més grans
    for num in range(len(grid), 0, -1):
        
        # Comprobem si pot haver un número en aquella posicó
        if satisfies(grid, x, y, num, top, bottom, left, right):
            
            grid[x][y] = num
            
            #delayed_print(grid, top, bottom, left, right, sleep_time=0.2)
           
            if skyscrapper_backtracking(grid, top, bottom, left, right, constraints):
                return True
            
            grid[x][y] = 0
            
            #delayed_print(grid, top, bottom, left, right, sleep_time=0.2)
    
    
    return False   


def skyscrapper(top, bottom, left, right):
    """
    Funció principal del problema. Rep quatre llistes corresponents als nombres que hi ha fora del tauler.
    
    Params
    ------
    :top, bottom, left, right: Els nombres de fora del tauler de la part superior, inferior, esquerra i dreta, respectivament.
    """
    
    
    # Inicialitzem una matriu de zeros
    grid = np.zeros((len(left), len(top)), dtype='int')
    
    
    prev_constraints = []
    constraints = []
    for i in range(len(grid)):
        
        if top[i] != 1:
            prev_constraints.append((top[i], i, "top"))
            
        else:
            constraints.append((top[i], i, "top"))
            
        if bottom[i] != 1:
            prev_constraints.append((bottom[i], i, "bottom"))
            
        else:
            constraints.append((bottom[i], i, "bottom"))
            
        if left[i] != 1:
            prev_constraints.append((left[i], i, "left"))      
            
        else:
            constraints.append((left[i], i, "left"))
            
        if right[i] != 1:
            prev_constraints.append((right[i], i, "right"))
             
        else:
            constraints.append((right[i], i, "right"))
    
    
    #we order by first value and if they are equal by the string of the third value and if they are equal by the int of the second value
    #we order first the ones that value on third value are top and left
    prev_constraints = sorted(prev_constraints, key=lambda x: (x[0], x[2], x[1]), reverse=True)


    print(prev_constraints)
    
    constraints += prev_constraints
    
    print(constraints)
    
    # Cridem a la funció que soluciona el problema mitjançant backtracking
    sol = skyscrapper_backtracking(grid, top, bottom, left, right, constraints)   
    
    # Mostrem el resultat en el cas que trobem una solució o mostrem un error en cas contrari.
    if sol:
        print(format_sky(np.array(grid), top, bottom, left, right))
    else:
        print('No solution found')





In [178]:
num_recursive_calls = 0
t0 = time.perf_counter()
skyscrapper([2,0],
            [0,0],
            [0,1],
            [0,0])
t1 = time.perf_counter()
print('Crides recursives:', num_recursive_calls)
print(f'Temps: {t1-t0:.2f}s')

[(2, 0, 'top'), (0, 0, 'bottom'), (0, 0, 'left'), (0, 0, 'right'), (0, 1, 'top'), (0, 1, 'bottom'), (0, 1, 'right')]
[(1, 1, 'left'), (2, 0, 'top'), (0, 0, 'bottom'), (0, 0, 'left'), (0, 0, 'right'), (0, 1, 'top'), (0, 1, 'bottom'), (0, 1, 'right')]
     2    
   +-------+
   | 1 | 2 |   
   +-------+
 1 | 2 | 1 |   
   +-------+
          
Crides recursives: 5
Temps: 0.00s


In [179]:
num_recursive_calls = 0
t0 = time.perf_counter()
skyscrapper([0,0,0],
            [3,0,0],
            [0,0,0],
            [0,0,1])
t1 = time.perf_counter()
print('Crides recursives:', num_recursive_calls)
print(f'Temps: {t1-t0:.2f}s')

[(0, 0, 'top'), (3, 0, 'bottom'), (0, 0, 'left'), (0, 0, 'right'), (0, 1, 'top'), (0, 1, 'bottom'), (0, 1, 'left'), (0, 1, 'right'), (0, 2, 'top'), (0, 2, 'bottom'), (0, 2, 'left')]
[(1, 2, 'right'), (0, 0, 'top'), (3, 0, 'bottom'), (0, 0, 'left'), (0, 0, 'right'), (0, 1, 'top'), (0, 1, 'bottom'), (0, 1, 'left'), (0, 1, 'right'), (0, 2, 'top'), (0, 2, 'bottom'), (0, 2, 'left')]
              
   +-----------+
   | 3 | 1 | 2 |   
   +-----------+
   | 2 | 3 | 1 |   
   +-----------+
   | 1 | 2 | 3 | 1 
   +-----------+
     3        
Crides recursives: 17
Temps: 0.00s


In [168]:
num_recursive_calls = 0
t0 = time.perf_counter()
skyscrapper([1,4,0,0],
            [0,1,0,0],
            [0,0,0,0],
            [0,1,0,0])
t1 = time.perf_counter()
print('Crides recursives:', num_recursive_calls)
print(f'Temps: {t1-t0:.2f}s')

[(1, 0, 'top'), (1, 1, 'bottom'), (1, 1, 'right'), (0, 0, 'bottom'), (0, 0, 'left'), (0, 0, 'right'), (4, 1, 'top'), (0, 1, 'left'), (0, 2, 'top'), (0, 2, 'bottom'), (0, 2, 'left'), (0, 2, 'right'), (0, 3, 'top'), (0, 3, 'bottom'), (0, 3, 'left'), (0, 3, 'right')]
     1   4        
   +---------------+
   | 4 | 1 | 3 | 2 |   
   +---------------+
   | 3 | 2 | 1 | 4 | 1 
   +---------------+
   | 2 | 3 | 4 | 1 |   
   +---------------+
   | 1 | 4 | 2 | 3 |   
   +---------------+
         1        
Crides recursives: 17
Temps: 0.00s


In [169]:
num_recursive_calls = 0
t0 = time.perf_counter()
skyscrapper([4,2,1,2,3],
            [1,4,3,2,2],
            [3,2,3,2,1],
            [3,4,1,2,2])
t1 = time.perf_counter()
print('Crides recursives:', num_recursive_calls)
print(f'Temps: {t1-t0:.2f}s')

[(1, 0, 'bottom'), (1, 2, 'top'), (1, 2, 'right'), (1, 4, 'left'), (4, 0, 'top'), (3, 0, 'left'), (3, 0, 'right'), (2, 1, 'top'), (4, 1, 'bottom'), (2, 1, 'left'), (4, 1, 'right'), (3, 2, 'bottom'), (3, 2, 'left'), (2, 3, 'top'), (2, 3, 'bottom'), (2, 3, 'left'), (2, 3, 'right'), (3, 4, 'top'), (2, 4, 'bottom'), (2, 4, 'right')]
     4   2   1   2   3
   +-------------------+
 3 | 2 | 3 | 5 | 4 | 1 | 3 
   +-------------------+
 2 | 1 | 5 | 4 | 3 | 2 | 4 
   +-------------------+
 3 | 3 | 4 | 2 | 1 | 5 | 1 
   +-------------------+
 2 | 4 | 2 | 1 | 5 | 3 | 2 
   +-------------------+
 1 | 5 | 1 | 3 | 2 | 4 | 2 
   +-------------------+
     1   4   3   2   2
Crides recursives: 47
Temps: 0.00s


In [170]:
num_recursive_calls = 0
t0 = time.perf_counter()
skyscrapper([2,1,4,3,2],
            [2,3,1,3,2],
            [2,1,3,5,2],
            [2,4,2,1,2])
t1 = time.perf_counter()
print('Crides recursives:', num_recursive_calls)
print(f'Temps: {t1-t0:.2f}s')

[(1, 1, 'top'), (1, 1, 'left'), (1, 2, 'bottom'), (1, 3, 'right'), (2, 0, 'top'), (2, 0, 'bottom'), (2, 0, 'left'), (2, 0, 'right'), (3, 1, 'bottom'), (4, 1, 'right'), (4, 2, 'top'), (3, 2, 'left'), (2, 2, 'right'), (3, 3, 'top'), (3, 3, 'bottom'), (5, 3, 'left'), (2, 4, 'top'), (2, 4, 'bottom'), (2, 4, 'left'), (2, 4, 'right')]
     2   1   4   3   2
   +-------------------+
 2 | 3 | 5 | 1 | 2 | 4 | 2 
   +-------------------+
 1 | 5 | 4 | 2 | 3 | 1 | 4 
   +-------------------+
 3 | 2 | 1 | 4 | 5 | 3 | 2 
   +-------------------+
 5 | 1 | 2 | 3 | 4 | 5 | 1 
   +-------------------+
 2 | 4 | 3 | 5 | 1 | 2 | 2 
   +-------------------+
     2   3   1   3   2
Crides recursives: 507
Temps: 0.01s


In [171]:
num_recursive_calls = 0
t0 = time.perf_counter()
skyscrapper([0,0,3,0],
            [0,2,2,0],
            [0,0,0,0],
            [0,0,0,0])
t1 = time.perf_counter()
print('Crides recursives:', num_recursive_calls)
print(f'Temps: {t1-t0:.2f}s')

[(0, 0, 'top'), (0, 0, 'bottom'), (0, 0, 'left'), (0, 0, 'right'), (0, 1, 'top'), (2, 1, 'bottom'), (0, 1, 'left'), (0, 1, 'right'), (3, 2, 'top'), (2, 2, 'bottom'), (0, 2, 'left'), (0, 2, 'right'), (0, 3, 'top'), (0, 3, 'bottom'), (0, 3, 'left'), (0, 3, 'right')]
             3    
   +---------------+
   | 4 | 1 | 2 | 3 |   
   +---------------+
   | 1 | 4 | 3 | 2 |   
   +---------------+
   | 3 | 2 | 4 | 1 |   
   +---------------+
   | 2 | 3 | 1 | 4 |   
   +---------------+
         2   2    
Crides recursives: 1181
Temps: 0.02s


<div class="alert alert-warning">

<h3>Expliqueu breument (màxim 5 línies) quina estratègia heu seguit.</h3>
    
<p>LA TEVA RESPOSTA AQUÍ</p>

</div>

<div class="alert alert-info">
<center>
  <h1>Entrega</h1>
</center>

<p>
L'entrega es farà a través del campus virtual. Teniu fins el dia <b>30 de Desembre a les 23:59</b> per entregar la pràctica. A la tasca que obrirem al campus, <b>només heu d'entregar aquest fitxer .ipynb</b>. El notebook ha de tenir com a nom:
</p>
<p>
    <code>Grup_Cognom_Nom_Enumeratius.ipynb</code>
</p>
    <br>
On 'Grup' és una sola lletra que identifica el grup on esteu matriculats. Els grups disponibles són: A, B, C, D, E, F.

És imprescindible que respecteu el format de les funcions i que el vostre codi estigui degudament comentat.<br><br>
</div>